# Task 1-2
Download three Polish models from the Huggingface repository.

In [1]:
from transformers import AutoTokenizer
from transformers import pipeline

/Users/piotrbialy/Projects/agh-data-science/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
herbert_model_name = "allegro/herbert-base-cased"
herbert_tokenizer = AutoTokenizer.from_pretrained(herbert_model_name)

herbert_fill_mask = pipeline(task="fill-mask", model=herbert_model_name, tokenizer=herbert_tokenizer)

In [3]:
roberta_model_name = "xlm-roberta-base"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)

roberta_fill_mask = pipeline(task="fill-mask", model=roberta_model_name, tokenizer=roberta_tokenizer)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
trelbert_model_name = "deepsense-ai/trelbert"
trelbert_tokenizer = AutoTokenizer.from_pretrained(trelbert_model_name)

trelbert_fill_mask = pipeline(task="fill-mask", model=trelbert_model_name, tokenizer=trelbert_tokenizer)

# Task 3
Devise a method to test if the langage model understands Polish cases.

In [5]:
import pandas as pd

In [6]:
def test_fill_mask(model, sentences: list[str], uppercase_mask: bool):
    if uppercase_mask:
        sentences = [sentence.replace("<mask>", "[MASK]") for sentence in sentences]
    
    series = {}

    for sentence in sentences:
        predictions = model(sentence)
        serie = [prediction["token_str"] for prediction in predictions]
        series[sentence] = serie

    return pd.DataFrame(series)

In [7]:
def test_polish_cases(model, uppercase_mask: bool = False):
    sentences = [
        "Kraków to dawna <mask> Polski.",
        "Jak grać w <mask> podwodnego?",
        "Przyglądałem się <mask> samochodom.",
        "Słyszę <mask> Metalliki.",
        "Ala pojechała z <mask> do weterynarza.",
        "Po koncercie wszyscy mówią o <mask> perkusisty.",
        "Powodzenia, <mask> magistrze!",
    ]

    return test_fill_mask(model, sentences, uppercase_mask)

In [8]:
test_polish_cases(herbert_fill_mask)

,Kraków to dawna <mask> Polski.,Jak grać w <mask> podwodnego?,Przyglądałem się <mask> samochodom.,Słyszę <mask> Metalliki.,Ala pojechała z <mask> do weterynarza.,Po koncercie wszyscy mówią o <mask> perkusisty.,"Powodzenia, <mask> magistrze!"
0,stolica,konia,tym,głosy,dzieckiem,pracy,panie
1,Stolica,tenisa,różnym,dźwięki,nim,śmierci,drogi
2,nazwa,hokeja,innym,muzykę,nią,karierze,Panie
3,stolicą,koła,wszystkim,śpiew,dziećmi,sukcesie,mój
4,stolicy,kole,też,głos,tym,roli,Drogi


In [9]:
test_polish_cases(roberta_fill_mask)

,Kraków to dawna <mask> Polski.,Jak grać w <mask> podwodnego?,Przyglądałem się <mask> samochodom.,Słyszę <mask> Metalliki.,Ala pojechała z <mask> do weterynarza.,Po koncercie wszyscy mówią o <mask> perkusisty.,"Powodzenia, <mask> magistrze!"
0,granica,psa,innym,o,dzieckiem,pracy,drogi
1,część,hokej,różnym,:,nią,roli,doktor
2,capital,poker,swoim,na,nami,śmierci,mój
3,karta,sport,tym,z,dziećmi,życiu,pani
4,strona,blackjack,nowym,od,nim,zmianie,kolega


In [10]:
test_polish_cases(trelbert_fill_mask)

,Kraków to dawna <mask> Polski.,Jak grać w <mask> podwodnego?,Przyglądałem się <mask> samochodom.,Słyszę <mask> Metalliki.,Ala pojechała z <mask> do weterynarza.,Po koncercie wszyscy mówią o <mask> perkusisty.,"Powodzenia, <mask> magistrze!"
0,stolica,konia,tym,głosy,nim,braku,panie
1,dzielnica,okrętu,takim,śpiew,psem,śmierci,Panie
2,Stolica,ptaka,swoim,głos,tym,pracy,drogi
3,część,świecie,ich,muzykę,nią,błędzie,pan
4,nazwa,wora,jego,hymn,dzieckiem,karierze,mój


# Task 4
Devise a method to test long-range relationships such as gender.

In [11]:
def test_verbs_relationships(model, uppercase_mask: bool = False):
    sentences = [
        "Andrzej jechał autem i <mask> przez telefon.",
        "Ala wczoraj grała w hokeja podwodnego i <mask> dwa gole.",
        "Dziecko wesoło <mask> ze szkoły i wyciagneło butlekę wody.",
        "Piorun trafił w dom i <mask> pożar.",
    ]

    return test_fill_mask(model, sentences, uppercase_mask)

In [12]:
test_verbs_relationships(herbert_fill_mask)

,Andrzej jechał autem i <mask> przez telefon.,Ala wczoraj grała w hokeja podwodnego i <mask> dwa gole.,Dziecko wesoło <mask> ze szkoły i wyciagneło butlekę wody.,Piorun trafił w dom i <mask> pożar.
0,rozmawiał,zdobyła,wyszło,wybuchł
1,rozmawialiśmy,straciła,poszło,spowodował
2,rozmawia,rzuciła,wychodziło,wywołał
3,mówił,strzelił,przyszło,przeżył
4,rozmawiali,wywalczyła,wróciło,wzbudził


In [13]:
test_verbs_relationships(roberta_fill_mask)

,Andrzej jechał autem i <mask> przez telefon.,Ala wczoraj grała w hokeja podwodnego i <mask> dwa gole.,Dziecko wesoło <mask> ze szkoły i wyciagneło butlekę wody.,Piorun trafił w dom i <mask> pożar.
0,rozmawia,dala,wychodzi,spowodował
1,mówił,miała,prosto,zrobił
2,tak,zdobył,wyjść,pali
3,powiedział,ma,wróci,wywoła
4,to,zdobywa,pochodzi,dostał


In [14]:
test_verbs_relationships(trelbert_fill_mask)

,Andrzej jechał autem i <mask> przez telefon.,Ala wczoraj grała w hokeja podwodnego i <mask> dwa gole.,Dziecko wesoło <mask> ze szkoły i wyciagneło butlekę wody.,Piorun trafił w dom i <mask> pożar.
0,rozmawiał,zdobyła,wyszło,spowodował
1,rozmawia,straciła,wróciło,wywołał
2,mówił,strzelił,przyszło,był
3,gada,miała,weszło,wybuchł
4,to,strzela,poszło,jest


# Task 5
Check if the model captures real-world knolwedge.

In [15]:
def test_real_world_knowledge(model, uppercase_mask: bool = False):
    sentences = [
        "Sport rozgrywany na dnie basenu z użyciem ciężkiego krążka oraz kijków to <mask> podwodny.",
        "Rzeka <mask> płynie przez Wrocław i ma źródła w Czechach.",
        "Jedynym metalem który w temperaturze pokojowej ma stan ciekły jest <mask>.",
        "Jadąc autostradą nie możesz przekraczać prędkości <mask> km/h.",
    ]

    return test_fill_mask(model, sentences, uppercase_mask)

In [16]:
test_real_world_knowledge(herbert_fill_mask)

,Sport rozgrywany na dnie basenu z użyciem ciężkiego krążka oraz kijków to <mask> podwodny.,Rzeka <mask> płynie przez Wrocław i ma źródła w Czechach.,Jedynym metalem który w temperaturze pokojowej ma stan ciekły jest <mask>.,Jadąc autostradą nie możesz przekraczać prędkości <mask> km/h.
0,sport,ta,woda,50
1,ruch,Odra,węgiel,40
2,taniec,Warta,tlen,100
3,bieg,najczęściej,olej,20
4,rower,często,aluminium,25


In [17]:
test_real_world_knowledge(roberta_fill_mask)

,Sport rozgrywany na dnie basenu z użyciem ciężkiego krążka oraz kijków to <mask> podwodny.,Rzeka <mask> płynie przez Wrocław i ma źródła w Czechach.,Jedynym metalem który w temperaturze pokojowej ma stan ciekły jest <mask>.,Jadąc autostradą nie możesz przekraczać prędkości <mask> km/h.
0,sport,ta,aluminium,100
1,ruch,która,grafit,80
2,tenis,Ta,metal,50
3,futbol,główna,sól,60
4,trening,",",olej,200


In [18]:
test_real_world_knowledge(trelbert_fill_mask)

,Sport rozgrywany na dnie basenu z użyciem ciężkiego krążka oraz kijków to <mask> podwodny.,Rzeka <mask> płynie przez Wrocław i ma źródła w Czechach.,Jedynym metalem który w temperaturze pokojowej ma stan ciekły jest <mask>.,Jadąc autostradą nie możesz przekraczać prędkości <mask> km/h.
0,sport,ta,woda,50
1,okręt,Odra,tlen,40
2,świat,Wisła,węgiel,10
3,statek,która,cukier,30
4,ruch,nie,olej,100


# Task 6
Check zero-shot learning capabilites of the models. 

In [19]:
def test_sentiments(model, uppercase_mask: bool = False):
    sentences = [
        "'Świetny koncert, zawsze mażyłem zobaczyć ich na żywo!' Ta wypowiedź ma zdecydowanie <mask> charakter.",
        "'Świetny koncert, zawsze mażyłem zobaczyć ich na żywo!' Ta wypowiedź zdecydowanie nie ma <mask> charakteru.",
        "'<mask> koncert, strata czasu i pieniędzy...' Ta wypowiedź ma zdecydowanie negatywny charakter.",
        "'ten dom wygląda strasznie, nie chciałbym w nim mieszkać.' Ta opinia zdecydowanie jest <mask>.",
        "'ten dom wygląda <mask>, nie chciałbym w nim mieszkać.' Ta opinia zdecydowanie nie jest pozytywna."
    ]

    return test_fill_mask(model, sentences, uppercase_mask)

In [20]:
test_sentiments(herbert_fill_mask)

,"'Świetny koncert, zawsze mażyłem zobaczyć ich na żywo!' Ta wypowiedź ma zdecydowanie <mask> charakter.","'Świetny koncert, zawsze mażyłem zobaczyć ich na żywo!' Ta wypowiedź zdecydowanie nie ma <mask> charakteru.","'<mask> koncert, strata czasu i pieniędzy...' Ta wypowiedź ma zdecydowanie negatywny charakter.","'ten dom wygląda strasznie, nie chciałbym w nim mieszkać.' Ta opinia zdecydowanie jest <mask>.","'ten dom wygląda <mask>, nie chciałbym w nim mieszkać.' Ta opinia zdecydowanie nie jest pozytywna."
0,pozytywny,sportowego,To,słuszna,źle
1,inny,oficjalnego,Dobry,prawdziwa,fatalnie
2,wyjątkowy,takiego,Ten,zła,strasznie
3,mocny,muzycznego,Jeden,pozytywna,pięknie
4,muzyczny,literackiego,Taki,dobra,świetnie


In [21]:
test_sentiments(roberta_fill_mask)

,"'Świetny koncert, zawsze mażyłem zobaczyć ich na żywo!' Ta wypowiedź ma zdecydowanie <mask> charakter.","'Świetny koncert, zawsze mażyłem zobaczyć ich na żywo!' Ta wypowiedź zdecydowanie nie ma <mask> charakteru.","'<mask> koncert, strata czasu i pieniędzy...' Ta wypowiedź ma zdecydowanie negatywny charakter.","'ten dom wygląda strasznie, nie chciałbym w nim mieszkać.' Ta opinia zdecydowanie jest <mask>.","'ten dom wygląda <mask>, nie chciałbym w nim mieszkać.' Ta opinia zdecydowanie nie jest pozytywna."
0,pozytywny,polskiego,To,moja,źle
1,wyjątkowy,żadnego,Na,najlepsza,dobrze
2,szczególny,dobrego,Ten,dobra,ładnie
3,swój,takiego,Super,ważna,świetnie
4,podobny,odpowiedniego,Za,ok,pięknie


In [22]:
test_sentiments(trelbert_fill_mask)

,"'Świetny koncert, zawsze mażyłem zobaczyć ich na żywo!' Ta wypowiedź ma zdecydowanie <mask> charakter.","'Świetny koncert, zawsze mażyłem zobaczyć ich na żywo!' Ta wypowiedź zdecydowanie nie ma <mask> charakteru.","'<mask> koncert, strata czasu i pieniędzy...' Ta wypowiedź ma zdecydowanie negatywny charakter.","'ten dom wygląda strasznie, nie chciałbym w nim mieszkać.' Ta opinia zdecydowanie jest <mask>.","'ten dom wygląda <mask>, nie chciałbym w nim mieszkać.' Ta opinia zdecydowanie nie jest pozytywna."
0,inny,żadnego,Ten,słuszna,źle
1,wyjątkowy,takiego,Jeden,prawdziwa,fatalnie
2,specyficzny,dobrego,To,zła,tragicznie
3,mocny,specjalnego,Kolejny,właściwa,super
4,dobry,swojego,Długi,uzasadniona,dziwnie


# Questions

## Which of the models produced the best results?

Herbert and trelbert are models that produced the best results. They were trained with only polish data, while reoberta was trained for many languages. 

## Was any of the models able to capture Polish grammar?

Yas, herbert and trelbert are able to capture, at least for top 2 predictions.. Herbert seems to be the best model in this task.

## Was any of the models able to capture long-distant relationships between the words?

Yas, herbert and trelbert are able to capture long-distant relationships, at least for top 2 predictions. Herbert seems to be the best model in this task.

## Was any of the models able to capture world knowledge?

Only herbert and trelbert have "Odra" in their top 5 predictions for second sentence in this task. For rest sentences in the task, they don't have correct answers. Roberta failed for every sentence in this task. Probably questions were too hard. Example with "... hokej podwodny." shows that these models cannot deduct if some phrase is very rare, even they have some hints, like  "z użyciem krążka oraz kijków". On the other hand, herbert and roberta suggested "hokej podwodny" in Task 3 in sentence "Jak grać w [MASK] podwodnego?".

## Was any of the models good at doing zero-shot classification?

Not at all, it seems that they have "positive bias" since they are good in classification of positive opinions, while they avoid pointing out bad opinions. 

## What are the most striking errors made by the models?

These models are pretty good at grammar, but they often don't have knowlege about the real world. 